# LLM Agents
This is the third notebook for the LLM section of Comp 255.

Sections:
* Is RAG an agent?
* Tool calling with AgentBot
* RAG + AgentBot
* (Optional) Fine-tuning results

In [ ]:
from llamabot import QueryBot, AgentBot, tool
from pathlib import Path

sft_model = "qwen2.5:1.5b"
agent_model = "qwen2.5:7b"

In [ ]:
# setting the data back up
doc1 = "Languages with statutory recognition in Papua New Guinea \
are Tok Pisin, English, Hiri Motu, and \
Papua New Guinean Sign Language"

doc2 = "The only language with statutory recognition in France \
is French"

# write these to a temporary file
with open("doc1.txt", "w") as f:
    f.write(doc1)

with open("doc2.txt", "w") as f:
    f.write(doc2)

In [ ]:
system_message = "You are a helpful assistant that answers questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]

query_completer = QueryBot(
    system_prompt=system_message,
    model_name = f"ollama_chat/{sft_model}",
    collection_name="documents",
    document_paths=doc_paths
)

In [ ]:
response = query_completer('Hi how are you today!')


Is this an agent? It did respond reasonably, but it seems to be pretty preoccupied with the RAG use-case.  An agent should have "agency"; it should be able to decide the right action to take.  Sometimes that might involve the RAG workflow, but other times it won't.

Retrieval can be thought of as a tool that the agent can choose to use.  One way to specify that is to create a function that does the retrieving.  Then, through natural language, we can describe the tool and the agent can decide if it makes sense to use it to respond to an input.  

Let's look at tool use more generally using `AgentBot`:

In [ ]:
# adapted from https://github.com/ericmjl/llamabot/blob/main/docs/tutorials/agentbot.md
# llamabot.tool decorator
@tool
def calculate_total_with_tip(bill_amount: float, tip_rate: float) -> float:
    if tip_rate < 0 or tip_rate > 1.0:
        raise ValueError("Tip rate must be between 0 and 1.0")
    return bill_amount * (1 + tip_rate)

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant with respect to restaurant bills.",
    functions=[calculate_total_with_tip, ],
    model_name=f"ollama_chat/{agent_model}",
    #model_name='gpt-4o'  
)

In [ ]:
# look at what is being provided to the model
print(calculate_total_with_tip.json_schema)

In [ ]:
bot.tools

In [ ]:
# system prompt has been extended with additional information
bot.decision_bot.system_prompt.content

In [ ]:
calculate_total_only_prompt = "My dinner was $2300 without tips. Calculate my total with an 18% tip."
response = bot(calculate_total_only_prompt, max_iterations=4)
print(response.content)

This will *usually* work.  The issue with small models (7 billion parameters in this case) is that they can sometimes output strange things, which will cause errors.  Sometimes the models can self-correct, but sometimes not.

The bot now could be said to have "agency"; it can decide whether to use a tool or just go ahead and respond normally.

In [ ]:
response = bot('Hello how are you?')
print(response.content)

Now, let's make the RAG workflow a tool.  Then we can use it to answer our question.

In [ ]:
# a bot within a bot - you can likely replace this with the LanceDB query itself
system_message = "You are a helpful assistant that can answer questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]
query_completer = QueryBot(
    system_prompt=system_message,
    model_name=f"ollama_chat/{agent_model}",
    collection_name="documents",
    document_paths=doc_paths,
)

@tool
def information_retrieval(query: str) -> str:
    query_completer(query)

# Create the bot
bot = AgentBot(
    system_prompt="You are my assistant and can retrieve information if needed.",
    functions=[information_retrieval],
    model_name=f"ollama_chat/{sft_model}",   
)

In [ ]:
response = bot("Hello how are you?")
print(response.content)

In [ ]:
response = bot("What are the national languages of Papua New Guinea")
print(response.content)

Again, we see some oddities, but we've managed to roughly get our RAG workflow to be an tool that the agent is able to make a decision to call with specific parameters.